In [1]:
%matplotlib inline

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np
from IPython.display import clear_output

import tqdm
import matplotlib.pyplot as plt

import re

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title
df = pd.read_csv('гороскопы мейла с фичами.csv', sep=';')

signs = ['Овен',	'Телец',	'Близнецы',	'Рак',	'Лев',	'Дева',	'Весы',	'Скорпион',	'Стрелец',	'Козерог',	'Водолей',	'Рыбы']
planets = {'Sun': 10,  'Mercury': 199, 'Venus': 299, 'Mars': 499, 'Jupiter': 599, 'Saturn': 699, 'Uranus': 799, 'Neptune': 899, 'Pluto': 999}
features = []
for k, v in planets.items():
  features += [k+'_x', k+'_y', k+'_z', k+'_vx', k+'_vy', k+'_vz', k+'_l', k+'_ry', k+'_rr']

In [ ]:
#@title
df = df.drop(['date',	'date_for_epoch'], axis=1)

In [4]:
#@title
data = pd.read_csv('/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/data/первые 5 слов коорд ohe emb.csv',sep=';')

In [5]:
#@title
data = data.dropna()

In [9]:
model_size = 50

In [4]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [8]:
model = Word2Vec.load("/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/data/word2vec_size_50_all_datasets.model")

In [10]:
kv_50 = WordEmbeddingsKeyedVectors.load("/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/data/WordEmbeddingsKeyedVectors_size_50.model")

In [11]:
data.columns[10:]

Index(['Sun_rr', 'Mercury_x', 'Mercury_y', 'Mercury_z', 'Mercury_vx',
       'Mercury_vy', 'Mercury_vz', 'Mercury_l', 'Mercury_ry', 'Mercury_rr',
       'Venus_x', 'Venus_y', 'Venus_z', 'Venus_vx', 'Venus_vy', 'Venus_vz',
       'Venus_l', 'Venus_ry', 'Venus_rr', 'Mars_x', 'Mars_y', 'Mars_z',
       'Mars_vx', 'Mars_vy', 'Mars_vz', 'Mars_l', 'Mars_ry', 'Mars_rr',
       'Jupiter_x', 'Jupiter_y', 'Jupiter_z', 'Jupiter_vx', 'Jupiter_vy',
       'Jupiter_vz', 'Jupiter_l', 'Jupiter_ry', 'Jupiter_rr', 'Saturn_x',
       'Saturn_y', 'Saturn_z', 'Saturn_vx', 'Saturn_vy', 'Saturn_vz',
       'Saturn_l', 'Saturn_ry', 'Saturn_rr', 'Uranus_x', 'Uranus_y',
       'Uranus_z', 'Uranus_vx', 'Uranus_vy', 'Uranus_vz', 'Uranus_l',
       'Uranus_ry', 'Uranus_rr', 'Neptune_x', 'Neptune_y', 'Neptune_z',
       'Neptune_vx', 'Neptune_vy', 'Neptune_vz', 'Neptune_l', 'Neptune_ry',
       'Neptune_rr', 'Pluto_x', 'Pluto_y', 'Pluto_z', 'Pluto_vx', 'Pluto_vy',
       'Pluto_vz', 'Pluto_l', 'Pluto_ry', 'Pluto_rr

In [12]:
len(kv_50.vocab)

3000

In [13]:
data.shape

(3730, 101)

In [14]:
len(data['start'].unique())

3016

In [15]:
def get_kv_vector(x):
  try:
    return np.array(kv_50.get_vector(x))
  except:
    return np.nan

In [16]:
data['sentence'] = data['start'].apply(get_kv_vector)

In [17]:
data['sentence'].isna().sum()

16

In [18]:
y_cols = ['Sun_x', 'Sun_y', 'Sun_z', 'Sun_vx', 'Sun_vy', 'Sun_vz',
       'Sun_l', 'Sun_ry', 'Sun_rr', 'Mercury_x', 'Mercury_y', 'Mercury_z',
       'Mercury_vx', 'Mercury_vy', 'Mercury_vz', 'Mercury_l', 'Mercury_ry',
       'Mercury_rr', 'Venus_x', 'Venus_y', 'Venus_z', 'Venus_vx', 'Venus_vy',
       'Venus_vz', 'Venus_l', 'Venus_ry', 'Venus_rr', 'Mars_x', 'Mars_y',
       'Mars_z', 'Mars_vx', 'Mars_vy', 'Mars_vz', 'Mars_l', 'Mars_ry',
       'Mars_rr', 'Jupiter_x', 'Jupiter_y', 'Jupiter_z', 'Jupiter_vx',
       'Jupiter_vy', 'Jupiter_vz', 'Jupiter_l', 'Jupiter_ry', 'Jupiter_rr',
       'Saturn_x', 'Saturn_y', 'Saturn_z', 'Saturn_vx', 'Saturn_vy',
       'Saturn_vz', 'Saturn_l', 'Saturn_ry', 'Saturn_rr', 'Uranus_x',
       'Uranus_y', 'Uranus_z', 'Uranus_vx', 'Uranus_vy', 'Uranus_vz',
       'Uranus_l', 'Uranus_ry', 'Uranus_rr', 'Neptune_x', 'Neptune_y',
       'Neptune_z', 'Neptune_vx', 'Neptune_vy', 'Neptune_vz', 'Neptune_l',
       'Neptune_ry', 'Neptune_rr', 'Pluto_x', 'Pluto_y', 'Pluto_z', 'Pluto_vx',
       'Pluto_vy', 'Pluto_vz', 'Pluto_l', 'Pluto_ry', 'Pluto_rr','Овен_cls', 'Телец_cls',
       'Близнецы_cls', 'Рак_cls', 'Лев_cls', 'Дева_cls', 'Весы_cls',
       'Скорпион_cls', 'Стрелец_cls', 'Козерог_cls', 'Водолей_cls', 'Рыбы_cls']

In [19]:
data = data.dropna()

In [20]:
data.head()

,Unnamed: 0,start,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,Venus_ry,Venus_rr,Mars_x,Mars_y,Mars_z,Mars_vx,Mars_vy,Mars_vz,Mars_l,Mars_ry,Mars_rr,Jupiter_x,Jupiter_y,...,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr,Овен_cls,Телец_cls,Близнецы_cls,Рак_cls,Лев_cls,Дева_cls,Весы_cls,Скорпион_cls,Стрелец_cls,Козерог_cls,Водолей_cls,Рыбы_cls,start_num,start_num_1,start_num_2,start_num_3,start_num_4,start_num_5,sentence
0,0,насыщенный интересный день за любые,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.545200,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,1.185614,0.006353,0.400402,0.142048,-0.020257,0.002517,-0.001505,0.000417,0.002457,0.425335,0.001847,1.671105,-4.844475,...,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819,1,0,0,0,0,0,0,0,0,0,0,0,[-0.20097116 -0.39993128 -0.38080287 -0.772286...,[-0.20097116 -0.39993128 -0.38080287 -0.772286...,[-0.24224697 -0.32531098 -0.29583368 -0.704362...,[-0.7106669 -0.23918846 -0.68920034 -0.818579...,[ 0.8948045 0.23784168 0.38576618 0.473802...,[ 0.05232921 -0.0726626 -0.6013601 -0.753021...,"[-1.45803964138031, 1.085614562034607, -2.6173..."
1,1,будьте практичны этот день может,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.551720,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,1.179244,0.006387,0.397954,0.143586,-0.020673,0.002379,-0.001572,0.000415,0.002446,0.423570,0.001682,1.657301,-4.833002,...,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824,1,0,0,0,0,0,0,0,0,0,0,0,[ 0.39644277 0.0206436 0.0720196 -0.075878...,[ 0.39644277 0.0206436 0.0720196 -0.075878...,[ 0.23707008 -0.21587744 0.03597016 -0.249976...,[ 0.11832371 -0.2798793 -0.7086075 -0.636874...,[-0.7106669 -0.23918846 -0.68920034 -0.818579...,[-0.23870714 -0.26901338 -0.12962128 -0.091806...,"[-3.3848729133605957, 3.2354576587677, -0.3398..."
2,2,благоприятный день если вы проявите,-0.918996,-0.386383,0.000023,0.006950,-0.015802,4.764212e-07,0.005758,0.996918,-0.000282,-0.560073,-0.489208,-0.041304,0.009234,0.012507,0.002104,0.004302,0.744790,-0.015276,-1.133805,0.299270,0.021828,-0.012420,-0.021963,0.001034,0.006774,1.172839,0.006421,0.395644,0.145190,-0.021087,0.002240,-0.001634,0.000413,0.002437,0.421971,0.001517,1.643766,-4.821399,...,0.108713,18.823040,-0.004627,28.499397,-5.857882,-0.565359,0.007509,-0.012693,-0.000076,0.168072,29.100688,0.009911,12.914777,-31.591994,-0.661335,0.009910,-0.015187,-0.000915,0.197154,34.136241,0.017822,1,0,0,0,0,0,0,0,0,0,0,0,[-2.89365470e-01 -4.67128426e-01 -6.83596671e-...,[-2.8936547e-01 -4.6712843e-01 -6.8359667e-01 ...,[-0.7106669 -0.23918846 -0.68920034 -0.818579...,[ 0.60200465 -0.55100614 0.12129481 -0.820373...,[ 0.33136016 -0.3066615 0.30393177 -0.716541...,[ 0.3615729 -0.41036066 -0.25291285 -0.054604...,"[-2.120943784713745, 1.8139744997024536, -1.01..."
3,3,первая половина дня удачна вам,-0.925808,-0.370523,0.000023,0.006673,-0.015916,6.468198e-07,0.005759,0.997200,-0.000281,-0.570154,-0.501349,-0.043293,0.010908,0.011757,0.001873,0.004392,0.760461,-0.016035,-1.121164,0.321016,0.020786,-0.012860,-0.021527,0.001051,0.006737,1.166401,0.006455,0.393475,0.146854,-0.021499,0.002099,

In [21]:
y = data[y_cols]

In [22]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') 
    return device

In [23]:
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [24]:
targets = df_to_tensor(y)

In [25]:
targets

tensor([[-9.0455e-01, -4.1775e-01,  2.3805e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-9.1191e-01, -4.0213e-01,  2.3554e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-9.1900e-01, -3.8638e-01,  2.3158e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-8.2391e-01, -5.5570e-01,  2.9905e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00],
        [-8.3369e-01, -5.4142e-01,  2.9420e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00],
        [-8.6150e-01, -4.9759e-01,  2.6956e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00]], device='cuda:0')

In [26]:
x_cols = ['sentence']

In [27]:
x_cols

['sentence']

In [28]:
x_cols = ['sentence']

In [29]:
data[x_cols].values

array([[array([-1.45803964,  1.08561456, -2.61732244,  0.31249788,  3.6280787 ,
       -2.61232924, -5.83150244,  1.05586815, -0.07457191, -0.01311547,
       -3.27058053,  0.32240713,  0.12138078,  3.97137547, -0.03816813,
        2.7501235 , -1.0767113 , -1.02983463,  0.51102436,  3.89514518,
       -2.77423477, -0.94586945,  2.07668042, -5.63616848,  2.45161843,
       -0.25273716, -3.86158609,  2.44153237, -2.23870206, -2.56828976,
       -1.30579281,  1.51927054,  1.36916149, -2.24898362, -4.16566277,
       -0.5056724 , -4.09440517, -6.62296152,  2.9077065 ,  0.91330194,
       -1.54240286, -0.16582775, -2.4099226 ,  1.12863398,  3.00414133,
        0.07415885, -1.11876321, -1.80352569, -1.36433876, -3.17386675])],
       [array([-3.38487291,  3.23545766, -0.33986923,  0.74598098,  3.06837368,
       -0.68221319, -3.49436283,  1.04541659, -2.86087251, -0.11893424,
       -4.55145311, -1.21118081,  0.75547391,  4.28840637, -0.92231905,
        2.72595596,  1.61777139,  0.3137697 ,

In [30]:
float_arr = np.vstack(data['sentence'].values).astype(np.float)

In [31]:
float_arr

array([[-1.45803964,  1.08561456, -2.61732244, ..., -1.80352569,
        -1.36433876, -3.17386675],
       [-3.38487291,  3.23545766, -0.33986923, ..., -3.07667875,
        -3.52305937, -2.8328054 ],
       [-2.12094378,  1.8139745 , -1.01357865, ..., -2.72285223,
        -1.25698185, -3.05837417],
       ...,
       [-3.44622231,  1.10938632, -1.69042397, ..., -2.87663651,
        -1.54794097, -3.5189817 ],
       [-4.07146454,  1.98398495, -0.74710482, ..., -0.11085257,
        -0.8191812 , -3.33398938],
       [-3.40640831,  1.08171248,  1.28549016, ..., -0.29742029,
        -2.58071733, -0.84794623]])

In [32]:
inps_1= torch.tensor(float_arr,dtype=torch.float32)

In [33]:
# float_arr_2 = np.vstack((data['start_num_2'].values)).astype(np.float)

In [34]:
# inps_2 = torch.tensor(float_arr_2,dtype=torch.float32)

In [35]:
# inps = torch.cat((inps_1,inps_2),dim=-1)

In [36]:
# len(data[x_cols[0]][0])

In [37]:
data.columns

Index(['Unnamed: 0', 'start', 'Sun_x', 'Sun_y', 'Sun_z', 'Sun_vx', 'Sun_vy',
       'Sun_vz', 'Sun_l', 'Sun_ry',
       ...
       'Козерог_cls', 'Водолей_cls', 'Рыбы_cls', 'start_num', 'start_num_1',
       'start_num_2', 'start_num_3', 'start_num_4', 'start_num_5', 'sentence'],
      dtype='object', length=102)

In [38]:
targets.shape

torch.Size([3714, 93])

In [39]:
inps_1.shape

torch.Size([3714, 50])

In [40]:
LATENT_DIM = 8

In [41]:
input_size = len(y_cols)

In [42]:
input_size

93

In [43]:
sent_len = 1

In [44]:
output_size = sent_len * model_size

In [45]:
output_size

50

In [187]:
gen = nn.Sequential(nn.Linear(input_size + LATENT_DIM, 128), nn.LeakyReLU(),
                    nn.Linear(128, 128), nn.LeakyReLU(),
                    nn.Linear(128, 64), nn.LeakyReLU(),
                    nn.Linear(64, 64), nn.LeakyReLU(),
                    nn.Linear(64, output_size)).cuda()
gen_opt = torch.optim.AdamW(gen.parameters(), lr=1e-2)
gen_scheduler = torch.optim.lr_scheduler.MultiStepLR(gen_opt, milestones=[30000, 50000], gamma=0.1)

In [188]:
disc = nn.Sequential(nn.Linear(input_size + output_size, 128), nn.LeakyReLU(),
                     nn.Linear(128, 64), nn.LeakyReLU(),
                     nn.Linear(64, 64), nn.LeakyReLU(),
                     nn.Linear(64, 1)).cuda()

disc_opt = torch.optim.AdamW(disc.parameters(), lr=1e-2)
disc_scheduler = torch.optim.lr_scheduler.MultiStepLR(disc_opt, milestones=[40000, 60000], gamma=0.1)

In [189]:
dataset = TensorDataset(inps_1, targets)

In [190]:
train, test = train_test_split(dataset, test_size=0.1, random_state=124124)

In [191]:
#train = dataset

In [192]:
TRAIN_BATCH_SIZE = 4

In [193]:
def sample_real_data(batch_size):
  while True:
    dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                             shuffle=True, drop_last=True)
    for batch in dataloader:
        yield batch


In [194]:
infinite_data = sample_real_data(TRAIN_BATCH_SIZE)

In [195]:
def sample_gen_data(y):
    noise = torch.randn(y.shape[0], LATENT_DIM, dtype=torch.float32, device="cuda")
    return torch.cat((gen(torch.cat((noise, y), dim=1)), y), dim=1)

In [196]:
def calc_gradient_penalty_classic(critic:torch.nn.Module,
                                  real_data:torch.Tensor,
                                  fake_data:torch.Tensor):
    assert real_data.shape == fake_data.shape
    alpha = torch.rand(real_data.shape[0], 1)
    alpha = alpha.expand(real_data.size()).cuda()

    interpolates = (alpha * real_data + ((1 - alpha) * fake_data)).cuda()
    interpolates = torch.autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = critic(interpolates)

    gradients = torch.autograd.grad(
      outputs=disc_interpolates, inputs=interpolates,
      grad_outputs=torch.ones(disc_interpolates.size(), device="cuda"),
      create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [197]:
LAMBDA = 1.
def train_disc(gan_type="WGAN-GP"):
   
    real_img, real_target = next(infinite_data) 
    real_img=real_img.to('cuda')
    real_target=real_target.to('cuda')
    y_gen = next(infinite_data)[1].to("cuda")
    gen_data = sample_gen_data(y_gen)
    real_data = torch.cat([real_img,real_target],dim = -1).to("cuda")

    if gan_type == "JS":
        logp_real_is_real = F.logsigmoid(disc(real_data))
        logp_gen_is_fake = F.logsigmoid(-disc(gen_data))
        disc_loss = -logp_real_is_real.mean() - logp_gen_is_fake.mean()
    elif gan_type == "WGAN-GP":
        disc_loss = disc(real_data).mean() - disc(gen_data).mean() + \
                  calc_gradient_penalty_classic(disc, real_data, gen_data)*LAMBDA
    else:
        raise ValueError("Unknown GAN type. Valid ones are: JS, WGAN-GP")

    disc_opt.zero_grad()
    disc_loss.backward()
    disc_opt.step()
    return disc_loss

In [198]:
def train_gen(gan_type="WGAN-GP"):
    real_data_y = next(infinite_data)[1].to("cuda")
    gen_data = sample_gen_data(real_data_y)

    if gan_type == "JS":
        logp_gen_is_real = F.logsigmoid(disc(gen_data))
        gen_loss = -logp_gen_is_real.mean()
    elif gan_type == "WGAN-GP":
        gen_loss = disc(gen_data).mean()
    gen_opt.zero_grad()
    gen_loss.backward()
    gen_opt.step()
    return gen_loss

In [5]:
RESULTS_PATH = '/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/GANs_results/GAN50_Lin_Lin_SentBySent_eval/'

In [200]:
TENSORBOARD_LOGDIR = RESULTS_PATH + "logs/"
summary_writer = SummaryWriter(log_dir=TENSORBOARD_LOGDIR)
local_writer = SummaryWriter(log_dir="./logs")
eval_writer = SummaryWriter(log_dir= TENSORBOARD_LOGDIR + "dists/")
eval_writer_local = SummaryWriter(log_dir= TENSORBOARD_LOGDIR + "./logs/dists/")

In [201]:
def sample_test_data(test):
  while True:
    dataloader = torch.utils.data.DataLoader(test, batch_size=len(test),
                                             shuffle=False, drop_last=True)
    for batch in dataloader:
        yield batch

In [89]:
test_data = sample_test_data(test)

In [202]:
def eval_dist(top = 3):
  gen.eval()
  xs_real,ys_real = next(test_data)
  xs_gen = sample_gen_data(ys_real.to('cuda'))[:,:output_size]
  dist = torch.cdist(xs_real.cuda(),xs_gen).mean().item()
  similars = []
  for x in xs_gen:
    d = kv_50.similar_by_vector(np.array(x.data.tolist()))
    ds = []
    for i in range(top):
      ds.append(d[i][1])
    similars.append(ds)
  a,b,c = np.array(similars).mean(axis = 0)
  gen.train()
  return dist, a,b,c

In [203]:
eval_dist()

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(16.50670623779297,
 0.31969824096848887,
 0.29777792929321206,
 0.29432836142919394)

In [204]:
def show_gen_text(sample_size = 1):  
  x_real,y_real = next(sample_real_data(sample_size))
  x_gen = sample_gen_data(y_real.to('cuda'))
  real = []
  generated = []
  for i in range(sample_size):
    print('Реальный текст:')
    real_text = kv_50.similar_by_vector(np.array(x_real[i]))[0][0]
    print(real_text)
    real.append(real_text)
    print('Сгенерированный текст:')
    b = np.array(x_gen[i][:50].tolist())
    gen_text = kv_50.similar_by_vector(b)
    for text in gen_text:
      print(text[0])
    generated.append(gen_text)
    print('\n')
  return real, generated

In [ ]:
show_gen_text()

Реальный текст:
вы хорошо знаете к чему
Сгенерированный текст:
переживаний и волнений будет много
будет много волнений и забот
появляется много новых идей каждая
дел и забот будет много
дел будет много но вряд
будет много интересных но довольно
появляется много новых идей но
появляется много хороших идей реализовать
привычные дела быстро начинают казаться
избежать волнений и переживаний едва




/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(['вы хорошо знаете к чему'],
 [[('переживаний и волнений будет много', 0.11546826362609863),
   ('будет много волнений и забот', 0.11428096890449524),
   ('появляется много новых идей каждая', 0.06568700075149536),
   ('дел и забот будет много', 0.0539109967648983),
   ('дел будет много но вряд', 0.050193216651678085),
   ('будет много интересных но довольно', 0.040719807147979736),
   ('появляется много новых идей но', 0.04030922055244446),
   ('появляется много хороших идей реализовать', 0.038583334535360336),
   ('привычные дела быстро начинают казаться', 0.031552694737911224),
   ('избежать волнений и переживаний едва', 0.026828650385141373)]])

In [ ]:
DISCRIMINATOR_ITERATIONS_PER_GENEREATOR = 5
VALIDATION_INTERVAL = 5000
SAVE_INTERVAL = 25000
EVAL_INTERVAL = 1000
gen.train()
for i in range(1000001):
    for _ in range(DISCRIMINATOR_ITERATIONS_PER_GENEREATOR):
        disc_loss_this_iter = train_disc()

    gen_loss_this_iter = train_gen()
    gen_scheduler.step()
    disc_scheduler.step()
    summary_writer.add_scalar("discriminator loss", disc_loss_this_iter,
                              global_step=i)
    summary_writer.add_scalar("generator loss", gen_loss_this_iter,
                              global_step=i)
    local_writer.add_scalar("discriminator loss", disc_loss_this_iter,
                              global_step=i)
    local_writer.add_scalar("generator loss", gen_loss_this_iter,
                              global_step=i)
    if i % EVAL_INTERVAL == 0 and i > 0:
      dist_1, dist_2,dist_3,dist_4 = eval_dist()
      eval_writer.add_scalar("Mean distance", dist_1,
                              global_step=i)
      eval_writer.add_scalar("Top 1 distance", dist_2,
                              global_step=i)
      eval_writer.add_scalar("Top 2 distance", dist_3,
                              global_step=i)
      eval_writer.add_scalar("Top 3 distance", dist_4,
                              global_step=i)
      eval_writer_local.add_scalar("Mean distance", dist_1,
                              global_step=i)
      eval_writer_local.add_scalar("Top 1 distance", dist_2,
                              global_step=i)
      eval_writer_local.add_scalar("Top 2 distance", dist_3,
                              global_step=i)
      eval_writer_local.add_scalar("Top 3 distance", dist_4,
                              global_step=i) 
    if i % VALIDATION_INTERVAL == 0 and i > 0:
      clear_output(True)
      print('Iter = ', i, 'D Loss = ', disc_loss_this_iter, 'G Loss = ', gen_loss_this_iter)
      show_gen_text(4)
    if i % SAVE_INTERVAL == 0 and i > 0:
      torch.save({'gen_state_dict': gen.state_dict(),
            'optimizer_state_dict': gen_opt.state_dict(),
            }, RESULTS_PATH + 'gen_'+ str(i))
      torch.save({'disc_state_dict': disc.state_dict(),
            'optimizer_state_dict': disc_opt.state_dict(),
            }, RESULTS_PATH + 'opt_'+ str(i))
      temp_df = pd.DataFrame(show_gen_text(20)).transpose()
      temp_df.columns = ['real', 'generated_sentence']
      temp_df.to_csv(RESULTS_PATH + 'texts_' + str(i) + '.csv')

      

Iter =  890000 D Loss =  tensor(-5.7950, device='cuda:0', grad_fn=<AddBackward0>) G Loss =  tensor(-0.1726, device='cuda:0', grad_fn=<MeanBackward0>)
Реальный текст:
первая половина дня складывается удачно
Сгенерированный текст:
неоднозначный день но точно не
напряженный но не безнадежный день
день складывается непросто не всегда
день не лишен трудностей но
день не лишен трудностей но
день не лишен трудностей но
день не лишен трудностей но
день не лишен трудностей но
день не лишен трудностей но
день не лишен трудностей но


Реальный текст:
насыщенный день который позволяет добиться
Сгенерированный текст:
старайтесь сохранять спокойствие сегодня это
старайтесь сохранять спокойствие сегодня это
трудно сохранять самообладание сегодня любой
нелегко сохранять спокойствие вы возмущаетесь
старайтесь сохранять самообладание хоть это
старайтесь сохранять самообладание если вы
старайтесь сохранять спокойствие даже если
нелегко сохранять спокойствие особенно в
нелегко сохранять спокойствие особен

In [6]:
%load_ext tensorboard

In [11]:
#test

In [ ]:
gen_load = nn.Sequential(nn.Linear(input_size + LATENT_DIM, 128), nn.LeakyReLU(),
                    nn.Linear(128, 64), nn.LeakyReLU(),
                    nn.Linear(64, 64), nn.LeakyReLU(),
                    nn.Linear(64, 64), nn.LeakyReLU(),
                    nn.Linear(64, output_size)).cuda()

In [ ]:
gen_load.load_state_dict(torch.load(RESULTS_PATH + 'gen_'+ str(i))['gen_state_dict']) # test

<All keys matched successfully>